---
# 1. Setting
---

In [10]:
# MLP 구현을 위한 모듈 선언
import torch
import torch.nn as nn
import torchvision.datasets as dataset
import torchvision.transforms as transform
from torch.utils.data import DataLoader
import torch.nn.functional as F
import numpy as np

In [11]:
# 폴더 경로 저장
datasetPath = "/content/drive/MyDrive/datatset"
parameterPath = '/content/drive/MyDrive/parameters'

In [12]:
# Iraining dataset 다운로드
cifar10_train = dataset.CIFAR10(root = datasetPath,                     
                            train = True,                           
                            transform = transform.ToTensor(),
                            download = True)
# testing dataset 다운로드
cifar10_test = dataset.CIFAR10(root = datasetPath,                      
                           train = False,                            
                           transform = transform.ToTensor(),                                                           
                           download = True)                          


Files already downloaded and verified
Files already downloaded and verified


---
# 2. Overfitting 방지
---

In [13]:
# 시계 방향 10도 Rotation
trans_rotation_right_10 = transform.RandomRotation((10,10))
rotation_data_right_10  = np.array(trans_rotation_right_10(torch.tensor(cifar10_train.data)))

# 시계 방향 20도 Rotation 수행
trans_rotation_right_20 = transform.RandomRotation((20,20))
rotation_data_right_20  = np.array(trans_rotation_right_10(torch.tensor(cifar10_train.data)))

# 시계 방향 30도 Rotation 수행
trans_rotation_right_30 = transform.RandomRotation((30,30))
rotation_data_right_30  = np.array(trans_rotation_right_10(torch.tensor(cifar10_train.data)))

# 반 시계 방향 10도 Rotation 수행
trans_rotation_left_10 = transform.RandomRotation((-10, -10))
rotation_data_left_10  = np.array(trans_rotation_right_10(torch.tensor(cifar10_train.data)))

# 반 시계 방향 20도 Rotation 수행
trans_rotation_left_20 = transform.RandomRotation((-20, -20))
rotation_data_left_20  = np.array(trans_rotation_right_10(torch.tensor(cifar10_train.data)))

# 반 시계 방향 30도 Rotation 수행
trans_rotation_left_30 = transform.RandomRotation((-30, -30))
rotation_data_left_30  = np.array(trans_rotation_right_10(torch.tensor(cifar10_train.data)))


In [14]:
cifar10_train.data = np.array(torch.cat((torch.tensor(cifar10_train.data), torch.tensor(rotation_data_right_10), torch.tensor(rotation_data_right_20), torch.tensor(rotation_data_right_30), torch.tensor(rotation_data_left_10), torch.tensor(rotation_data_left_20), torch.tensor(rotation_data_left_30)),0))

In [15]:
cifar10_train.targets = cifar10_train.targets*7

---
# 3. 모델
---

In [16]:
# MLP 모델 정의
class MLP(nn.Module):
  def __init__(self): 
    super(MLP, self).__init__()                                       
    self.fc1 = nn.Linear(in_features=3072, out_features=340)                                                                                                  
    self.fc2 = nn.Linear(in_features=340, out_features=340)                    
    self.fc3 = nn.Linear(in_features=340, out_features=10)           
    self.relu = nn.ReLU()                                    
    self.dropout = nn.Dropout(0,2)
    self.bn = nn.BatchNorm1d(340)

    torch.nn.init.xavier_normal_(self.fc1.weight.data)
    torch.nn.init.xavier_normal_(self.fc2.weight.data)
    torch.nn.init.xavier_normal_(self.fc3.weight.data)

  def forward(self, x): 
                                   
    x = x.view(-1,3*32*32)                                             
                                                                
    y = self.relu(self.bn(self.fc1(x)))                                     
    y = self.relu(self.bn(self.fc2(y)))                                                                            
    y = self.dropout(y)
    y = self.fc3(y)                                                    
    return y    

In [17]:
# MNIST 분류를 위한 MLP 모델 학습
batch_size = 142                                                    
learning_rate = 0.1                                                
training_epochs = 16                                             
loss_function = nn.CrossEntropyLoss()                            
                                                                     
network = MLP()                                                 
optimizer = torch.optim.SGD(network.parameters(), lr = learning_rate, momentum=0.9) 

data_loader =   DataLoader(dataset = cifar10_train,                   
                           batch_size = batch_size,                 
                           shuffle = True,          
                           drop_last = True
                           )                                            

testdata_loader = DataLoader(dataset = cifar10_test,
                             batch_size=batch_size,
                             shuffle=False,
                             drop_last = True
                             )


---
# 4. 학습
---

In [18]:
# 모델 학습
for epoch in range(training_epochs):                                  
  avg_cost = 0                                                       
                                                                     
                                                                      
  total_batch = len(data_loader)                                    

  for img, label in data_loader:                                      
                                                                     
    pred = network(img)                                               
  
    loss = loss_function(pred, label)                              
    optimizer.zero_grad()                                            
    loss.backward()                                                  
    optimizer.step()                                                  

    avg_cost +=loss/total_batch                                       

  print('Epoch: %d Loss = %f' %(epoch+1, avg_cost))                  
print('Learning finished')                                           

Epoch: 1 Loss = 1.352333
Epoch: 2 Loss = 0.963359
Epoch: 3 Loss = 0.715658
Epoch: 4 Loss = 0.540844
Epoch: 5 Loss = 0.422009
Epoch: 6 Loss = 0.340585
Epoch: 7 Loss = 0.291048
Epoch: 8 Loss = 0.251585
Epoch: 9 Loss = 0.218712
Epoch: 10 Loss = 0.194330
Epoch: 11 Loss = 0.178574
Epoch: 12 Loss = 0.160340
Epoch: 13 Loss = 0.149575
Epoch: 14 Loss = 0.135997


KeyboardInterrupt: ignored

---
# 5. 저장
---

In [27]:
torch.save(network.state_dict(), parameterPath + 'mlp_cifar10.pth')

In [28]:
new_network = MLP()
new_network.load_state_dict(torch.load(parameterPath+'mlp_cifar10.pth')) 

<All keys matched successfully>

---
# 6. 평가
---

In [29]:
#Training된 Network 성능 확인
with torch.no_grad():

    img_test = torch.tensor(np.transpose(cifar10_test.data, (0, 3, 1, 2)))/255
    label_test = torch.tensor(cifar10_test.targets)

    prediction = network(img_test)

    correct_prediction = torch.argmax(prediction, 1) == label_test
    accuracy = correct_prediction.float().mean()
print("Accuracy:" , accuracy.item())

Accuracy: 0.5263000130653381
